In [28]:
import pandas as pd
import numpy as np
import datetime, time

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from kneed import KneeLocator

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [2]:
df=pd.read_csv('data/ens_2014.csv')


In [3]:
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [ ]:
results = []

depart = time.time()  
for day in day_order:
    print(f'jour {day} : {datetime.datetime.now().strftime("%H:%M:%S")}')
    duree_jour = time.time()
    df_day=df[df['day_of_week']==day]
    print(f'jour {day}, {df_day.shape[0]} lignes')
    for i in range(24):
        df_hour = df_day[df_day['hour']==i] 
        coords = df_hour[['Lat', 'Lon']].values
        inertias = []
        silhouette_scores = []

        #Je limite le nombre de cluster de 1 à 9 pour éviter trop de changement 
        K = range(2, 10)

        for k in K:
            kmeans = KMeans(n_clusters=k, random_state=42, n_init = 'auto')
            kmeans.fit(coords)
            inertias.append(kmeans.inertia_)

        try:
            kneedle = KneeLocator(K, inertias, curve="convex", direction="decreasing")
            coude = kneedle.knee
        except Exception as e:
            coude = None

        if coude is None or np.isnan(coude):
            coude = 3  # par défaut je mets 3 car ça risque correspondre aux périodes avec peu de données
        else:
            coude = int(coude)

        kmeans = KMeans(n_clusters=coude, random_state=42, n_init='auto')
        labels = kmeans.fit_predict(coords)
        silhouette = silhouette_score(coords, labels)

        results.append({
            'jour': day,
            'heure': i,
            'cluster': coude,
            'silhouette': silhouette,
            'labels': labels.tolist(),
            'indexes': df_hour.index.tolist()
        })
    duree_journée = time.time()-duree_jour
    print(f"⏱️ Durée totale : {duree_journée:.2f} secondes\n")

# DataFrame final
df_res = pd.DataFrame(results)
df_res.to_csv('data/df_resultat_clusters.csv', index=False)
display(df_res.head())


In [ ]:
df_res=pd.read_csv('data/df_resultat_clusters.csv')
df_res

In [ ]:
df_res.info()

In [15]:
df_res.shape[0]

168

In [ ]:
print(df_res.iloc[0]['indexes'])
print(type(df_res.iloc[0]['indexes']))

Le code ne fonctionne pas car indexes et labels sont des chaines de caractères.

Je vais recharger df_res et faire en sorte que labels et indexes soient des listes

In [17]:
df_res = pd.read_csv(
    "data/df_resultat_clusters.csv",
    converters={'labels': eval, 'indexes': eval}
)

In [19]:
df_clusters_list = []

for _, row in df_res.iterrows():
    indexes = row['indexes']
    labels = row['labels']

    # Vérification que ceux sont bien des listes sinon suivant
    if not isinstance(indexes, list) or not isinstance(labels, list):
        continue
    # verification qu'il y a bien des valeurs et que c'est la même longueur entre labels et indexes
    if len(indexes) == 0 or len(labels) == 0 or len(indexes) != len(labels):
        continue

    # Sélection des points correspondants dans df
    subset = df.loc[indexes].copy()

    # Ajout des informations de clustering
    subset['cluster'] = labels
    subset['jour'] = row['jour']
    subset['heure'] = row['heure']
    subset['best_k'] = row['cluster']
    subset['silhouette'] = row['silhouette']

    df_clusters_list.append(subset)

df_clusters = pd.concat(df_clusters_list, ignore_index=True)


In [ ]:
df_clusters.head()

In [ ]:
jour = "Monday"
df_jour = df_clusters[df_clusters['jour'] == jour]

fig = px.scatter_map(
    df_jour, lat="Lat", lon="Lon", color=df_jour['cluster'].astype(str),
    animation_frame="heure", zoom=9, 
    title=f"Clusters K-Means par heure – {jour}"
)
fig.show()

In [30]:
jours = df_clusters['jour'].unique()
jours

array(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday',
       'Sunday'], dtype=object)

In [ ]:
for jour in jours:
    df_jour = df_clusters[df_clusters['jour'] == jour]

    fig = px.scatter_map(
        df_jour, lat="Lat", lon="Lon", color=df_jour['cluster'].astype(str),
        animation_frame="heure", zoom=9,
        title=f"Clusters K-Means par heure – {jour}",
        height=700,
        width=1200 
    )
    fig.show()